# 시작하기 전에

---

현재까지의 학습 내용으로는 훈련 세트에서 모델을 훈련하고 테스트 세트에서 모델을 평가했다. 테스트 세트에서 얻은 정확도와 점수를 보고 실전에서 모델의 성능을 기대하는 식이다. 그런데 테스트 세트를 사용해 성능을 확인하다 보면 테스트 세트에 맞추게 되는것인데, 따지고보면 훈련 세트와 테스트 세트에 과대적합 되는 것과 같다.

따라서 테스트 세트로 일반화 성능을 올바르게 사용하려면 한 테스트 세트를 사용하지 말아야 한다. 모델을 만들고 나서 마지막에 딱 한번만 사용하는 것이 좋다.

그렇다면 바로 전에 학습했던 Decision Tree의 max_depth 매개변수를 사용한 하이퍼 파라미터 튜닝을 어떻게 할 수 있을까? 게다가 결정 트리는 테스트 해 볼 매개변수가 많다.

# 검증 세트

---

테스트 세트를 사용하지 않으면 모델이 과대적합인지 과소적합인지 판단하기 어렵다. 테스트 세트를 사용하지 않고 이를 측정하는 간단한 방법은 훈련세트를 또 나누는 것이다.

테스트 세트를 사용하지 않고 이를 측정한 간단한 방법은 훈련 세트를 또 나누는 것인데, 이 데이터를 검증 세트(validation set)라고 부른다.

이 방법이 너무 단순해서 이상하게 들릴 수 있겠지만 실제로 많이 사용하는 방법이다.

1절에서 전체 데이터 중 20%를 테스트 세트로 만들고 나머지 80%를 훈련 세트로 만들었다. 이 훈련 세트 중에서 다시 20%를 떼어 내어 검증 세트로 만든다.

---

보통 20% ~ 30%의 테스트 세트를 떼어와서 검증세트를 만들지만 상황에 따라 테스트 세트의 크기가 크다면 몇 %만 떼어와도 전체 데이터를 대표하는데 문제가 없다.

In [2]:
# 이전 절에서 사용했던 데이터를 다시 불러와 검증 세트 만들어 보기

import pandas as pd

df = pd.read_csv('https://bit.ly/wine_csv_data')

In [3]:
data = df[['alcohol', 'sugar', 'pH']]

target = df[['class']]

target

,class
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
6492,1.0
6493,1.0
6494,1.0
6495,1.0


In [4]:
data = data.to_numpy()

target = target.to_numpy()

test_data와 train_data를 나누는 경우에

train_test_split을 통해서 나누었는데

test_size 매개변수를 이용해서 비율을 정해줄 수 있다.
만일 test_size = 0.2로 정해준다면

|80%|20%|
|---|----|
|train_set|test_set|

와 같이 나타낼 수 있다.

In [5]:
#test_size를 통해서 데이터 나누기
from sklearn.model_selection import train_test_split
print(data.shape)
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

test_input.shape

(6497, 3)


(1300, 3)

test_size 매개변수를 통해서 데이터를 80%의 훈련 세트와 20%의 테스트 세트로 나누었다.

교차검증이 포함되어 있는 데이터의 비율은 훈련6 : 검증2 : 테스트2 이다.

|60%|20%|20%|
|---|---|---|
|train_set|validation_set|test_set|


In [6]:
#test_size를 통해서 validation_set 만들기

sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state = 42)

In [7]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [8]:
# 전 장에서 학습했던 dt를 이용해서 모델 생성 후 평가하기

from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state = 42)

dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))

print(dt.score(val_input, val_target))

print(dt.score(test_input, test_target))

0.9971133028626413
0.864423076923077
0.8569230769230769


위에서 훈련, 검증, 테스트 세트의 정확도를 모두 확인해본 결과, 훈련 세트에 과대적합 되어있는 것을 확인할 수 있다.

따라서 max_depth를 비롯한 parameters를 조정해서 더 좋은 모델을 찾아야 한다.

그 전에 검증 세트에 고나해서 조금 더 알아야 할 것이 있다.

# 교차 검증(cross validation)

---

검증 세트를 만드느라 훈련 세트가 줄어 들었다. 보통 많은 데이터를 훈련에 사용할 수록 좋은 모델이 만들어 진다.

훈련세트에 많은 데이터를 보존하기 위해서 검증세트에 적은 데이터를 주게되면, 검증세트에 대한 정확도가 불안정할 수 있다.

이런 상황에서 교차 검증(cross validation)을 이용하면 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 사용할 수 있다.

교차 검증은 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복한다. 그다음 이 점수를 평균하여 최종 검증 점수를 반환한다.

## 3-폴드 교차 검증

---

훈련 세트를 세 부분으로 나눠서 교차 검증을 수행하는 것을 3-폴드 교차 검증이라고 하며 틍칭 k-폴드 교차검증(k-fold cross validation)이라고 하며 k-겹 교차검증이라고도 부른다.

과정은 다음과 같다.

|train_set|train_set|validation_set|
|:------------:|:------:|:-------:|
|train|train|evaluate|
|||
|||
|train_set|validation_set|train_set|
|train|evaluate|train|
|||
|||
|validation_set|train_set|train_set|
|evaluate|train|train|

훈련세트는 모델 훈련에 쓰이고 각 부분의 검증세트를 평균하여 최종 검증 점수를 얻는다.

보통은 5-폴드 교차검증이나 10-폴드 교차검증을 많이 사용하는데 이렇게 하면 데이터의 80 ~ 90%까지 훈련에 사용할 수 있기 떄문이다

## 교차 검증 적용

---

사이킷런에서 cross_validate(), cross_val_score() 함수를 사용해서

위에서 만들었던 dt모델의 교차검증을 진행하여 accuracy를 확인할수 있다.

In [9]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)

print(scores)


{'fit_time': array([0.01533389, 0.0156951 , 0.01696754, 0.01249909, 0.01665473]), 'score_time': array([0.00217724, 0.00190759, 0.00196671, 0.00257158, 0.00235629]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


위에서 설명한 함수는 cross_validate()함수와 그 전신이 되는 cross_val_score()함수이다.

cross_validate()함수의 경우에는

fit_time, score_time, test_score키를 가진 딕셔너리를 반환한다.

처음 2개의 키는 각각 모델을 훈련하는 시간과 검증하는 시간을 의미한다.

각 키마다 5개의 숫자가 담겨있는데 cross_validate()함수는 기본적으로 5-폴드 교차검증을 수행하기 때문에 각 검증세트 폴드에 대한 모델 평가 점수이다.

cv매개변수에서 폴드 수를 바꿀 수도 있다.

cross_val_score()의 경우에는 함수의 결과 중에서 test_score 값만 반환한다.

여기에서 말하는 test_score은 테스트세트의 정확도가 아닌 검증세트의 정확도이다.

In [10]:
import numpy as np


# 5폴드로 나눈 검증세트들의 정확도 평균
print(np.mean(scores['test_score']))


0.855300214703487


교차 검증을 수행하면 입력한 모델에서 얻을 수 있는 최상의 검증 점수를 가늠해볼 수 있다. 한가지 주의할 점은 cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않는다.

앞서 train_test_split() 함수로 전체 데이터를 섞은 후 훈련 세트를 준비했기 때문에 따로 섞을 필요가 없었지만, 만약 교차 검증을 할 떄 훈련 세트를 섞으려면 분할기(splitter)를 지정해야 한다.

사이킷런의 분할기는 교차 검증에서 폴드를 어떻게 나눌지 결정한다.

사이킷런의 분할기는 교차 검증에서 폴드를 어떻게 나눌지 결정해준다. cross_validate()함수는 기본적으로 회귀(Regression) 모델일 경우 KFold 분할기를 사용하고 분류(Classifier) 모델일 경우 타깃 클래스를 골고루 나누기 위해 StratifiedKFold를 사용한다. 즉 앞서 수행한 교차 검증은 다음 코드와 동일하다.

In [11]:
from sklearn.model_selection import StratifiedKFold
import pandas as pd



scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())

print(np.mean(scores['test_score']))


print(pd.DataFrame(scores))

0.855300214703487
   fit_time  score_time  test_score
0  0.017176    0.003248    0.869231
1  0.014762    0.002570    0.846154
2  0.013154    0.004345    0.876805
3  0.015046    0.002267    0.848893
4  0.013998    0.002180    0.835419


분할기로 섞은 후에 10-폴드 교차 검증을 수행한다.

In [12]:
spliter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)

scores = cross_validate(dt, train_input, train_target, cv = spliter)

print(np.mean(scores['test_score']))

0.8574181117533719


현재까지는 Classifier model에 대해서 StratifierKFold 를 사용했지만 KFold의 사용법도 동일하다.

# 하이퍼 파라미터 튜닝

---

머신러닝 모델이 학습하는 파라미터를 모델 파라미터라고 하는데, 반면에 모델이 학습할 수 없어서 지도해줘야 하는 파라미터를 하이퍼 파라미터라고 한다.

사이킷런과 같은 머신러닝 라이브러리를 사용할 때 이런 하이퍼파라미터는 모두 클래스나 메서드의 매개변수로 표현된다.

ex) random_state, epoch, cv, r, alpha ...

하이퍼파라미터를 튜닝하는 작업은 다음과 같다. 먼저 라이브러리가 제공하는 기본값을 그대로 사용해 모델을 훈련한다.

>먼저 라이브러리가 제공하는 기본값을 그대로 사용해 모델을 훈련한다.
>> 그다음 검증 세트의 점수나 교차 검증을 통해서 매개변수를 조금씩 바꿔본다.
>>> 모델마나 제공하는 매개변수를 바꿔가면서 모델을 훈련하고 교차검증을 수행한다.

하이퍼파라미터를 튜닝하지 않고 자동으로 수행하는 기술을 AutoML이라 부른다.

<!-- 사람의 개입 없이 하이퍼파라미터 튜닝을 자동으로 수행하는 기술을 "AutoML"이라 부른다. -->

DecisionTreeClassifier 클래스를 사용했을 때 매개변수들을 알아본 적이 있었다.

그중에서 max_depth, min_samples_split 등등의 매개변수 중에서

max_depth에 대한 최적의 값을 찾았다고 가정해보자.

max_depth 그렇다면 max_depth를 최적의 값을 고정하고 다른 매개변수의 최적값을 찾으면 참 좋겠지만 그렇지 않은데  
min_samples_split 파라미터의 값이 달라지면 max_depth의 최적값도 변한다.
>노드가 나눠지는 최소 샘플 개수가 바뀌면 max_depth에도 영향이 가기 때문

게다가 매개변수가 많아지면 문제는 더 복잡해진다.  
단순 반복문으로 파라미터의 최적값을 찾는다고 하면 다중반복문을 사용해야하는데 그로 인한 시간이 너무 오래 걸릴 수 있다.

따라서 사이킷런에서 제공하는 그리드 서치(Grid Search)를 사용한다.

사이킷런의 GridSearchCV 클래스는 친절하게도 하이퍼파라미터 참색과 교차 검증을 한번에 수행한다. 별도로 cross_validate() 함수를 호출할 필요가 없다.




## GridSearchCV

---
GridSearchCV의 파라미터는 다음과 같다.

* param_grid  
하이퍼파라미터 튜닝을 위해 사용될 파라미터들을  dictionmary 형태로 만들어 놓는다.

* scoring  
각 하이퍼파라미터 조합을 모델평가에 사용되는 성능 지표이다.

* cv  
cross validation, 폴드라는 개념으로 교차검증을 위해 분할되는 수를 의미한다.

* verbose  
iteration 수행 시 수형 결과 메세지를 출력  
verbose = 0(출력 안함), 1(출력), 2(파라미터 별 메세지 출력)

* n_jobs  
GridSearch 병렬 실행에 사용할 CPU 코어의 갯수, -1일 경우 모든 코어 실행

* refit  
데이터세트로 찾은 최적화된 하이퍼파라미터를 사용해 최적모델 재훈련 여부 결정


적용 과정

1. 필요한 라이브러리 모델 가져오기

2. 데이터세트 불러오기

3. 조정하려는 분루자의 인스턴스 만들기  
ex) GS = GridSearch()

4. gridsearchCV에서 사용할 하이퍼파라미터 변수들 정의

5. GridSearch 객체만들고 파라미터 지정하기

6. GridSearchCV를 이용하여 GridSearchCV 수행

7. 수행 후 최적의 하이퍼 파라미터 확인하기

In [13]:
# Decision Tree에서 min_impurity_decrease 매개변수의 최적값 찾기

#GridSearchCV 임포트하고 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 만들기

from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)



결정 트리 글래스의 객체를 생성하자마자 바로 전달하였고 sklearn의 다른 모델과 비슷하게 fit을 해준다.  
이 메서드를 호출하면 그리드 서치 객체는 결정 트리 모델의 min_impurity를 params에 담겨있는 value에 따라 값을 바꿔가며 총 5번 실행한다.

GridSearchCV의 cv매개변수 기본값은 5이다.(5-fold cross validation)

따라서 매개변수가 바뀔때마다, 폴드에 따른 validation set가 달라질 때마다 모델이 다르기 때문에 총 25개의 모델을 훈련하는 셈이다.

n_jobs을 -1로 지정해서 모든 코어을 사용한다.

In [14]:
# 그리드서치 실행하기
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

cross validation에서 최적의 하이퍼파라미터를 찾으면 전체 훈련 세트로 모델을 다시 만들어야 하는데, sklearn의 gridsearch는 훈련이 끝나면 25개의 모델 중에서 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련세트에서 다시 모델을 훈련한다.  
> 아직 식견이 좁아서 잘 모르지만 시간이 오래걸린다는 단점 말고는 괴물같다.

점수가 가장 높은 모델은 gs 객채의 best_estimator_ 속성에 저장되어있다. 이 모델을 일반 결정트리 처럼 똑같이 사용할 수 있다.


> 가장 높은 성능을 냈던 하이퍼파라미터의 값은 best_params_ 속성  
> 가장 높은 정확도의 값은 best_score_ 속성에서 확인할 수 있다.



In [15]:
print("Best Hyperparameters: ", gs.best_params_)
print("Best Score: ", gs.best_score_)

# cv_results_속성에서 각 매개변수에서 수행한 교차 검증의 평균 점수 확인

print(gs.cv_results_['mean_test_score'])

Best Hyperparameters:  {'min_impurity_decrease': 0.0001}
Best Score:  0.8681929740134745
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [16]:
# 가장 높은 검증 점수를 가지는 모델을 dt선언

dt = gs.best_estimator_

print(dt.score(train_input, train_target))

print(dt.score(test_input, test_target))



0.9615162593804117
0.8653846153846154


mean_test_score을 사용해서 5번의 교차 검증으로 얻은 점수를 확인했을 때 첫번째 값이 가장 큰데 numpy에서 argmax()를 이용하면 가장 큰 값의 인덱스를 추출할 수 있다. 그다음 이 인덱스를 사용해 params 키에 저장된 매개변수를 출력할 수 있다.

In [17]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])

print(gs.cv_results_['mean_test_score'][best_index])


0.8681929740134745


위 과정을 정리해보자면

- 먼저 탐색할 매개변수를 dictionary형태로 지정한다.  
-그다음 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾는다.   
이 조합은 그리드 서치 객체에 저장됨  
- 그리드 서치는 최상의 매개변수에서 교차 검증에서 사용한 훈련 세트가 아닌 전체 훈련 세트를 사용해 최종 모델을 훈련한다.  
이 모델도 그리드 서치 객체에 저장된다.



다음으로 파라미터들에 대한 dictionary를 만들고 이에 따라서 다시 GridSearch를 진행한다.

In [18]:
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
           'max_depth' : range(5, 20, 1),
          'min_samples_split' : range(2, 100, 10)
          }

일반적인 python에서 제공하는 range의 경우 소수점이 있는 실수형인 float에 대해서 지원하지 않기 때문에 numpy에서 제공하는 arange함수를 이용한다.

각각의 경우에 따라서 1개씩 모델을 만들고 5-폴드 교차검증으로 다른 모델을 또 만들기 때문에 모델의 수는 엄청나게 많아진다.  
> 아래 경우들의 배열 길이들과, 숫자를 곱하면 모델 갯수가 된다.
>> min_impurity_deacrease의 경우 arange 0.001 - 0.0009까지 0.0001씩 증가한 배열
>> max_depth의 경우 5부터 19까지 1씩 증가한 배열
>> min_samples_split의 경우 2부터 99까지 10씩 증가한 배열
>> cv의 매개변수 값만큼 생기는 폴드의 갯수(default = 5)

In [19]:
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [20]:
# 가장 높은 성능을 냈던 하이퍼파라미터 값 확인하기

print(gs.best_params_)

# 가장 높은 정학도의 교차검증 점수 확인하기

print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731


# 랜덤 서치(random_search)

---

매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있다.

>특히나 소수점까지 가는 경우에는 얼마나 많은 모델을 학습해야할지 감이 안잡히기 때문에 ex) np.arange(0.0001, 0.1, 0.0001)  

이런 경우에는 GridSearchCV의 수행시간이 오래 걸릴 수 있는데 이럴 때 랜덤 서치를 사용하면 좋다.

랜덤 서치에는 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 확률 분포객체를 전달한다.
> 아직까지는 무슨말인지 잘 모르겠음

In [21]:
# scipy 라이브러리를 이용해서 확률 분포 클래스 임포트하기

from scipy.stats import uniform, randint

scipy의 stats 서브 패키지에 있는 uniform과 randint 클래스는 모두 주어진 범위에서 고르게 값을 뽑는다.  
이를 ***균동 분포에서 샘플링한다*** 라고 한다.  

randint는 int(정수)를 뽑고, uniform은 float(실수)을 뽑는다.


In [29]:
# 0 ~ 10 의 범위를 가지는 randint객체를 만들고 10개의 숫자를 샘플링

rgen = randint(0, 10)

print(rgen.rvs(10))

rgen.rvs(1000)

np.unique(rgen.rvs(1000))

# return_counts = True로 하여 각 원소의 갯수 세기
np.unique(rgen.rvs(1000), return_counts = True)

[2 5 4 4 8 2 0 6 7 9]


(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 96, 102, 104,  98, 100,  94,  99, 105,  98, 104]))

In [33]:
# 0 ~ 1 사이의 범위를 가지는 uniform

ugen = uniform(0, 1)

print(ugen)

# np.unique(ugen.rvs(1000), return_counts = True)

0부터 1사이의 소수점아래로 8자리까지 나타나기 때문에 제대로 겹치기 어렵다.

randint와 uniform은 하나의 난수 발생기와 같다. 랜덤 서치에 randint와 uniform 클래스 객체를 넘겨주고 총 몇 번을 샘플링해서 최적의 매개변수를 찾으라고 명령할 수 있다.  
샘플링 횟수는 아무래도 하드웨어가 허락하는 범위 내에서 최대한으로 가져가는게 좋다.

랜덤서치를 사용해서 다시 params의 딕셔너리를 만들어보자

여기에서 기존에 했던 매개변수 3개에 min_samples_leaf 매개변수를 탐색 대상에 추가한다.

이 매개변수는 리프 노드가 되기 위한 최소 샘플의 갯수로, 어떤 노드가 분할하여 만들어진 자식 노드의 샘플 수가 이 값보다 작을 경우 분할하지 않는다.

전체 값의 범위는 같지만 그 중에서 매개변수의 값을 랜덤으로 추출해서 진행한다.

In [42]:
params = {
    'min_impurity_decrease' : uniform(0.0001, 0.001),
    'max_depth' : randint(20, 50),
    'min_samples_split' : randint(2, 25),
    'min_samples_leaf' : randint(1, 25),
}

In [43]:
# random_search를 위해서 RandomizedSearchCV import
from sklearn.model_selection import RandomizedSearchCV

# n_iter를 통해서 randint와 uniform의 rvs를 정해줄 수 있다.
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params, n_iter = 100, n_jobs = -1, random_state = 42)


In [44]:
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7d38df1c18a0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7d38de3c38b0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7d38df1c04f0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7d38df1c3130>},
                   random_state=42)

In [45]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [47]:
dt = gs.best_estimator_


print(dt.score(train_input, train_target))
print(dt.score(test_input, test_target))


0.8928227823744468
0.86


원래 훈련세트보다 테스트세트의 점수가 낮은것이 일반적이다.
> 테스트세트의 점수가 더 높으면 과대 적합의 가능성이 존재하기 때문.

# 고찰

---

이번 장의 경우 전 장에서 배웠던 의사결정트리의 성능을 끌어올리기 위해서 교차검증과 그리드서치, 랜덤서치를 진행하여 문제를 해결하고자 하였다.

우선 의사결정트리의 성능을 끌어올리려먼 하이퍼파라미터의 튜닝으로 최적화된 값을 찾아내는 것인데,  

훈련세트가 데이터의 상당수를 차지한다면 훈련세트 전체에 대해서 매개변수를 맞추게 되고 그렇다면 훈련세트에만 맞춰진 과대적합된 모델을 얻을 수 있다.  

때문에 훈련세트에서 데이터를 또 소분하여 ***검증세트***를 만들어 문제를 해결할 수 있었다.

그러나 검증세트의 경우 훈련세트를 다 대변하지 못하고 샘플링 편향이 발생할 가능성이 있다. 때문에 ***교차검증***을 이용하셔 진행하였다.

교차검증은 훈련세트를 균일하게 나누고, 한 부분을 선택하여 검증세트로 활용한다.  

전체 데이터를 N등분하여 나누어진 한 덩어리를 폴드라 부른다.
> 일반적으로 5-폴드 / 10-폴드 교차검증을 진행한다.  
>> 5-폴드의 경우 데이터를 20%를, 10-폴드의 경우 데이터를 10%를 검증세트로 나눈다.

교차 검증은 검증 세트를 떼어내어 평가하는 과정을 반복하여서 검증 세트의 정확도를 평균하여 최종 검증 점수를 얻는다.

> 3-폴드 교차검증의 경우
>> 훈련 - 훈련 - 검증  
>> 훈련 - 검증 - 훈련  
>> 검증 - 훈련 - 훈련  
각 3가지 경우에 대해서 훈련세트 66%는 훈련을 진행하고 검증세트는 테스트를 진행한다.


* train_test_split을 통해서 섞여져 있는 데이터를 따로 섞을 필요가 없다면
sklearn.model_selection에 있는 cross_validate 교차 검증 함수를 통해서 교차검증을 진행할 수 있다.

* 훈련세트를 섞고 교차검증을 진행하려면 sklearn.model_selection에 있는 StratifiedKFold를 통해서 골고루 재분배하고 cross_validate()함수에 넣어서 결과를 확인할 수 있다.

분배 및 교차검증까지 한 뒤에 하이퍼파라미터 튜닝을통해서 의사결정트리의 성능을 향상시킬 수 있었다.

---

그리드 서치와 랜덤 서치가 있는데  

그리드 서치는 매개변수를 주어진 범위 내에서 각각의 경우에 따라 하이퍼파라미터 탐색과 교차검증을 한번에 수행해준다.    
매개변수의 범위가 넓고 간격이 좁다면 그만큼 경우가 많아지고 모델이 많아지면서 실행시간이 길다는 단점이 있다.
- sklearn.model_selection에서 GridSearchCV를 임포트해서 사용할 수 있다.


랜덤 서치는 매개변수를 주어진 범위 내에서 확률 분포 객체를 전달하고 그걸 균등 분포에서 샘플링하여 훈련을 진행한다.

- scipy.stats에서 uniform, randint를 임포트하여 주어진 범위 내에서 균등 분포에서 샘플링 가능하다.

* sklearn.model_selection 에서 RandomizedSearchCV를 임포트하여 사용 가능하다.

>최적의 매개변수를 찾는것은 아마도 지루한 과정일 수 있다.  
각 모델에 따라서 매개변수의 값이 바뀌고 여러 매개변수가 최적으로 떨어지는 값을 찾기 어렵기 때문이다.
각 매개변수가 딱 맞아떨어지는 그 지점을 수동으로는 찾기 어렵기 때문에  
하이퍼파라미터 튜닝에 있어서 그리드 서치와 랜덤 서치는 효율적이다.

# 마무리

## scikit-learn

---

* ***cross_validate()***  
  교차 검증을 수행하는 함수이다.
  - estimator  
  fit을 구현하는 객체  
  ex) DecisionTreeClassifier, LinearRegression...  
  ---

  - x  
  The data to fit,  
  input데이터가 되겠다.
  ---

  - y  
  The target variable to try predict in the case of supervised learning  
  지도학습에서 예측을 시도할 변수로 target데이터가 되겠다.  
  ---

  - groups
  Group labels for the samples uesd while spliiting the dataset into  
  train/test set. Only used in conhjunction with a "Group"  
  > default = None  

    - dataset을 훈련/ 테스트 세트로 분할할 때 사용되는 샘플의 그룹 레이블  

 ---

 - scoring  
 Strategy to evaluate the performance of the cross-validated model on the test set.  
 > default = None  

   - 매개변수 검중에 사용할 평가 지표를 선택한다. 기본적으로 Regression_model은
   결정계수를 의미하는 'r2', Classifier_model은 정확도를 의미하는 'accuracy'가 된다.  

 ---

 - cv  
 Determines the cross-validation splitting strategy.  
 >default = 5

  - 교차 검증 폴드 수나 splitter객체를 지정할 수 있으며, 분류일 때는 StratifiedKFold를 사용하고 회귀일 경우에는 KFold 클래스를 사용한다.








In [1]:
!wget https://bit.ly/fruits_300_data -O fruits_300.npy

--2024-01-15 21:30:51--  https://bit.ly/fruits_300_data
Resolving bit.ly (bit.ly)... 67.199.248.11, 67.199.248.10
Connecting to bit.ly (bit.ly)|67.199.248.11|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/rickiepark/hg-mldl/raw/master/fruits_300.npy [following]
--2024-01-15 21:30:51--  https://github.com/rickiepark/hg-mldl/raw/master/fruits_300.npy
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rickiepark/hg-mldl/master/fruits_300.npy [following]
--2024-01-15 21:30:51--  https://raw.githubusercontent.com/rickiepark/hg-mldl/master/fruits_300.npy
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... conne